## Control Web with Selenium

- import
- Chrome 시작 with chrome driver
- Chrome 시작 (최신버전)
- Close & Quit
- Chrome Open with Chrome Options
- 요소 선택과 상호작용
- 요소 선택과 상호작용(2) - Select/Alert/...
- 요소 선택과 상호작용(3) - Table 분해
- window/frame 전환
- 팝업창 제거
- Wait
- Window 크기 조절
- 페이지 이동
- execute_script
<br><br>
-------------------------------------------------------------
<div style="text-align: right"> : 2025-01-31 </div>

### import

In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from pywinauto.keyboard import send_keys
from selenium.webdriver.common.alert import Alert
from selenium.webdriver.support.select import Select

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import json

import time, os

### Chrome 시작 with chrome driver

In [ ]:
chrome_ver = 102 # 맨 앞자리
path = ""
url = "https://"

driver = webdriver.Chrome(f"{path}\chromedriver_{chrome_ver}\chromedriver.exe",)
driver.get(url)

In [ ]:
# 만일 여러 VDI가 하나의 프로그램을 공유할 경우
# chromedriver 버전차이 때문에 오류가 생길 수 있다.
# 그때 사용하는 코드
# 해당 폴더에 사용하는 모든 driver 폴더를 넣어놓고 시작

import os

list_dir = os.listdir(os.getcwd())
driver_list = []
for x in list_dir:
    if "chromedriver" in x:
        driver_list.append(x)

# driver_list.pop(0)

from selenium import webdriver

for chromedriver in driver_list:
    try:
        driver = webdriver.Chrome(f"{os.getcwd()}\{chromedriver}\chromedriver.exe",)
    except:
        print(chromedriver, ": 실패")
        pass
    else:
        driver.get("https://onspace.sk.com/")
        print(chromedriver, ": 성공")
        break

### Chrome 시작 (최신버전)

In [ ]:
# 예전 버전과는 다르게 이제는 chrome driver를 따로 다운받지 않아도 된다.
# 인터넷 접속이 제한된 상황에서는 한계가 있을 수 있다.

from selenium.webdriver import Chrome

driver = Chrome()

### Close & Quit

In [ ]:
# close
driver.close()

# quit
driver.quit()

### Chrome Open with Chrome Options

In [ ]:
import json

# 아래 코드는 "인쇄" 시 "PDF로 저장"으로 디폴트 설정된 코드 # 저장경로 고정
chrome_options = webdriver.ChromeOptions()
appState = {"recentDestinations": [{"id": "Save as PDF", "origin": "local", "account": ""}],
            "selectedDestinationId": "Save as PDF",
            "version": 2,
            "isHeaderFooterEnabled": False,
            "isLandscapeEnabled": True,
            "scalingType": 3,
            "scaling": "141"}
prefs = {'printing.print_preview_sticky_settings.appState': json.dumps(appState),
        'savefile.defau1t_directory': r"~~저장 경로"}
chrome_options.add_experimental_option('prefs', prefs)
chrome_options.add_argument('kiosk-printing')

driver = webdriver.Chrome(f"{os.getcwd()}\chromedriver_{chrome_ver}\chromedriver.exe", options=chrome_options)

### 요소 선택과 상호작용

In [ ]:
driver.find_element(by=By.CSS_SELECTOR, value="~~") # id -> #~ / class -> .~ / ...
driver.find_element(by=By.XPATH, value="~~")
driver.find_element(by=By.ID, value="~~")
driver.find_element(by=By.CLASS_NAME, value="~~")
# , ...By.LINK_TEXT By.TAG_NAME By.NAME By.PARTIAL_LINK_TEXT

# 상호작용
'''
    .click()
    .send_keys()
    .clear()
    .text()
    ...etc.

'''

# 특수키 입력
from selenium.webdriver.common.keys import Keys
input_box = "" # ex) driver.find_element(By.~~, "~~~")
input_box.send_keys(Keys.Enter)

'''
    Keys.ENTER / TAB / SHIFT / CONTROL / ALT / SPACE / BACKSPACE / DELETE / LEFT / RIGHT / UP / DOWN / F1~F12 / ...

'''

### 요소 선택과 상호작용(2) - Select/Alert/...

In [ ]:
from selenium.webdriver.common.alert import Alert
from selenium.webdriver.support.select import Select

# select
option = ""
select = Select(driver.find_element(by=By.ID, value="~~"))
select.select_by_visible_text(option)

select = Select(driver.find_element(by=By.ID, value="~~"))
select.select_by_value("G")

# alert
alert = Alert(driver) # 알림창
alert.accept() # "확인"
alert.dismiss() # "취소"
alert.text # 텍스트 요소 가져오기

### 요소 선택과 상호작용(3) - Table 분해

In [ ]:
# 예시 코드를 통한 설명

col_name_list = [] # table 열 이름 목록
# time.sleep(3)
table = driver.find_element(by=By.CSS_SELECTOR, value= "~~~")
thead = table.find_element(by=By.TAG_NAME, value="thead")
tr = thead.find_element(by=By.TAG_NAME, value="tr")
for th in tr.find_elements(by=By.TAG_NAME, value="th"):
    col_name_list.append(th.text)

# 열 이름 목록에서 조회 원하는 열 이름의 인덱스 찾기
num_idx = col_name_list.index("번호")
Qty_sum_idx = col_name_list.index("수량 합계")
sum_qty_idx = col_name_list.index("SUM Qty")

tbody = table.find_element(by=By.TAG_NAME, value="tbody")
trows = tbody.find_elements(by=By.TAG_NAME, value="tr")

prod_num_list = []

for tr in trows: # table 열 단위로 탐색
    tdata = tr.find_elements(by=By.TAG_NAME, value="td")

    if "없" in tdata[0].text: # table 없을 때
        print("조회된 결과가 없습니다.")
        break
    
    prod_num = tdata[num_idx].text
    startQty_sum = tdata[Qty_sum_idx].find_element(by=By.TAG_NAME, value="input").get_attribute('value')

    if prod_num:
        pass
    else:
        print("~~")

### window/frame 전환

In [ ]:
# window 전환
def SwitchWindow(self): # 팝업창 <-> 기존창 driver 변경 함수 # 팝업창 하나일 때
    driver.switch_to.window(driver.window_handles[-1])

n = 3 # n번째 창
driver.switch_to.window(driver.window_handles[n])

# 프레임 전환
driver.switch_to.default_content()
driver.switch_to.frame(driver.find_element(by=By.XPATH, value='~~~'))

### 팝업창 제거

In [ ]:
main = driver.window_handles
# 팝업창 종료
for handle in main:
    if handle != main[0]:
        driver.switch_to.window(handle)
        driver.close()
driver.switch_to.window(main[0])

### Wait

In [ ]:
# 강제 대기
time.sleep(3)

# Implicitly Wait
# 명령어를 줬을 때 다음 웹페이지가 넘어올 때까지 기다리라는 뜻
# => s초 이내에 페이지가 뜨면 바로 넘어감
# => s초가 넘어가면 웹페이지 로딩 여부와 관계없이 다음 명령어를 실행
driver = webdriver.Chrome('chromedriver.exe')
driver.impolicitly_wait(10)

# Explicitly Wait
# 해당 요소가 나타날 때까지 기다리라는 뜻
# => s초 이내에 페이지가 뜨면 바로 넘어감
element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#ID")))

### Window 크기 조절

In [ ]:
# 최대화
driver.maximize_window()

# 최소화
driver.minimize_window()

# 사이즈 조절
driver.set_window_size(1250, 1410)

# 위치 조절
driver.set_window_position(1300, 0)

### 페이지 이동

In [ ]:
# 이전 페이지로 이동
driver.back()

# 다음 페이지로 이동
driver.forward()

# 새로고침
driver.refresh()

### execute_script

In [ ]:
# 자바 스크립트로 되어있는 페이지를 조작할 때 사용하는 방법
driver.execute_script("document.getElementById('WD79).cilck()")
driver.execute_script("document.getElementById('WD79).value ='"+ "~~" + "';")
driver.execute_script("document.getElementById('WD75).cilck()")
driver.execute_script("document.getElementById('WD75).value ='"+ "~~" + "';")